# Problem 

The question provides a discrete dynamical system in the form:
$\bar{x}_{k+1}=f\left(\bar{x}_{k}\right)$

Where:
$\bar{x}_{k}=\begin{bmatrix} x_k & v_k \end{bmatrix}$ is the state at time step k,

$\bar{x}_{k+1}=\begin{bmatrix} x_{k+1} & v_{k+1} \end{bmatrix}$ is the next state

Input-output pairs: $\left(\bar{x}_k, \bar{x}_{k+1}\right)$

The target is to approximate the unknown function f using a regression tree, and use it to:
Predict $\bar{x}_{k+1}$ from $\bar{x}_k$ & Simulate the system forward in time.




In [1]:
import numpy as np

# Generate synthetic data
n_samples = 100
xk_list = []
vk_list = []
x_next_list = []
v_next_list = []

xk = 0.0
vk = 10.0

for _ in range(n_samples):
    xk_list.append(xk)
    vk_list.append(vk)
    x_next = xk + 0.1 * vk # Simulate a dataset to show how a vehicle moves at a constant velocity over time
    v_next = 10.0  # Constant velocity

    x_next_list.append(x_next)
    v_next_list.append(v_next)

    xk = x_next  # Update state
    vk = v_next

# Create feature and label arrays
X = np.column_stack((xk_list, vk_list))           # Current state: (xk, vk)
Y = np.column_stack((x_next_list, v_next_list))   # Next state:    (xk+1, vk+1)

## Regression Tree
Need to be modified to handle multi-dimensional outputs, as the state is a vector [x, v]

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # This can now be a vector (multi-output)

class RegressionTree:
    def __init__(self, max_depth=None, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape

        # Stopping conditions
        if num_samples <= self.min_samples_leaf or (self.max_depth is not None and depth >= self.max_depth):
            return Node(value=np.mean(y, axis=0))

        best_feature, best_threshold = self._find_best_split(X, y)
        if best_feature is None:
            return Node(value=np.mean(y, axis=0))

        left_indices = X[:, best_feature] <= best_threshold
        right_indices = ~left_indices

        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return Node(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape
        best_feature, best_threshold, best_sse = None, None, float('inf')

        for feature in range(num_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if sum(left_mask) < self.min_samples_leaf or sum(right_mask) < self.min_samples_leaf:
                    continue

                sse = self._compute_sse(y[left_mask], y[right_mask])
                if sse < best_sse:
                    best_feature, best_threshold, best_sse = feature, threshold, sse

        return best_feature, best_threshold

    def _compute_sse(self, y_left, y_right):
        def sse(y):
            return np.sum((y - np.mean(y, axis=0)) ** 2)

        return sse(y_left) + sse(y_right)

    def predict(self, X):
        return np.array([self._predict_sample(x, self.root) for x in X])

    def _predict_sample(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._predict_sample(x, node.left)
        else:
            return self._predict_sample(x, node.right)

## Training
Train a regression tree to learn the function f from data: 
$f\left(x_k, v_k\right) \approx \left(x_{k+1}, v_{k+1}\right)$

In [3]:
# Initialize and train the model
tree = RegressionTree(max_depth=100, min_samples_leaf=2)
tree.fit(X, Y)

For predicting the vector the tree has been modified  so that each node can store and predict a vector. Once trained the tree is ready for predicting the next stage.

In [4]:
# Predict the next state for a sample
test_state = np.array([[5.0, 10.0]])  # (xk, vk)
predicted_next_state = tree.predict(test_state)

print("Current state:", test_state[0])
print("Predicted next state:", predicted_next_state[0])

Current state: [ 5. 10.]
Predicted next state: [ 5. 10.]


Simulate multiple time steps by iteratively feeding the output as the next input.

In [5]:
# Simulate using learned model
state = np.array([0.0, 10.0])  # Initial state: x=0, v=10
trajectory = [state.copy()]

for _ in range(20):
    state = tree.predict([state])[0]
    trajectory.append(state.copy())

trajectory = np.array(trajectory)

# Print trajectory
for i, s in enumerate(trajectory):
    print(f"Step {i}: x = {s[0]:.2f}, v = {s[1]:.2f}")

Step 0: x = 0.00, v = 10.00
Step 1: x = 2.00, v = 10.00
Step 2: x = 2.00, v = 10.00
Step 3: x = 2.00, v = 10.00
Step 4: x = 2.00, v = 10.00
Step 5: x = 2.00, v = 10.00
Step 6: x = 2.00, v = 10.00
Step 7: x = 2.00, v = 10.00
Step 8: x = 2.00, v = 10.00
Step 9: x = 2.00, v = 10.00
Step 10: x = 2.00, v = 10.00
Step 11: x = 2.00, v = 10.00
Step 12: x = 2.00, v = 10.00
Step 13: x = 2.00, v = 10.00
Step 14: x = 2.00, v = 10.00
Step 15: x = 2.00, v = 10.00
Step 16: x = 2.00, v = 10.00
Step 17: x = 2.00, v = 10.00
Step 18: x = 2.00, v = 10.00
Step 19: x = 2.00, v = 10.00
Step 20: x = 2.00, v = 10.00
